In [ ]:
%matplotlib inline

#- import required python libraries
import netCDF4
import sys, os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import glofrim.glofrim_lib as glib
from os.path import join, isfile, abspath, dirname, basename, normpath
from configparser import ConfigParser
from glofrim.glofrim_lib import configread

In [ ]:
from glofrim import Glofrim, PCR, LFP
cbmi = Glofrim()

In [ ]:
#- provide file with local paths to model objects
env_fn = r'../../environment.env'
config = configread(env_fn)
root_dir = config.get('models', 'root_dir')

In [ ]:
#- convert model configs to dictionary
_cf = ConfigParser(inline_comment_prefixes=('#'))
env = glib.configread(abspath(env_fn), encoding='utf-8', cf=_cf)
glib.config2dict(env)

In [ ]:
#- provide file with coupling settings
# config_fn = join(root_dir, 'glofrim_PCR2CMF.ini')
# config_fn = join(root_dir, 'glofrim_PCR2DFM.ini')
# config_fn = join(root_dir, 'glofrim_WFL2DFM.ini')
config_fn = join(root_dir, 'glofrim_PCR2LFP.ini')
config_fn

In [ ]:
#- provide locations where to extract simulated discharge
x_obs_coord = 10.89 # set to coords of Neu-Darchau
y_obs_coord = 53.23

In [ ]:
#- create coupled GLOFRIM model and initialize configs
try:
    cbmi.initialize_config(config_fn, env_fn=env_fn)
except Exception as e:
    print e
    sys.exit('configs could not be initialized')

In [ ]:
#- provide start and end date
t_start = datetime(2000,1,1)
t_end = datetime(2000,1,3)
cbmi.set_start_time(t_start)
cbmi.set_end_time(t_end)
try:
    t_start == cbmi.get_start_time()
    t_end == cbmi.get_end_time()
except:
    sys.exit('start or end time differ with set_var and get_var')

In [ ]:
#- create coupled GLOFRIM model and initialize models
try:
    cbmi.initialize_model()
except Exception as e:
    print e
    sys.exit('models could not be initialized')

In [ ]:
#- get corresponding indices for models
obs_ind_PCR = cbmi.index(x_obs_coord, y_obs_coord, 'PCR', in1d=False)
print obs_ind_PCR
obs_ind_LFP = cbmi.index(x_obs_coord, y_obs_coord, 'LFP', in1d=True)
print obs_ind_LFP

In [ ]:
#- update models until end time is reached
#- also, read out discharge data at specified observation point
try:
    i = 0
    while i < 10:
#     while t_start < t_end:
        cbmi.update()
        q_sim_PCR = cbmi.get_value_at_indices('PCR.discharge', obs_ind_PCR)
        q_sim_LFP = cbmi.get_value_at_indices('LFP.Qx', obs_ind_DFM) 
        print q_sim_PCR, q_sim_LFP
        i += 1
except Exception as e:
    print e
    sys.exit('something is going wrong in updating - please check!')

In [ ]:
#- finalize all models
try:
    cbmi.finalize()
except Exception as e:
    print e
    sys.exit('models could not be finalized')